In [1]:
from transformers import BertTokenizerFast
import pandas as pd

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer_bert = BertTokenizerFast.from_pretrained('kykim/bert-kor-base')

In [4]:
train = pd.read_csv('/root/share/data/train.csv')
train = train[train['data'].notnull()]

In [5]:
train_data = tokenizer_bert(train['data'].tolist(), truncation=True, padding=True)

In [7]:
len(train_data.data['input_ids'][0])

512

In [9]:
del train_data

In [10]:
del tokenizer_bert